In [ ]:
%pip install numpy tensorflow matplotlib language-tool-python

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import language_tool_python
import pandas as pd

In [56]:
# Cargar el conjunto de datos desde un archivo CSV (ajusta el nombre del archivo según sea necesario)
df = pd.read_csv('oraciones.csv')

# Mostrar las primeras filas para verificar la carga correcta
print(df.head(), "\n")
print(df.columns)

                                            CORRECTA   
0  La puesta de sol sobre el horizonte pintaba el...  \
1  Después de una larga jornada de trabajo, disfr...   
2  El aroma fresco de las flores primaverales imp...   
3  Con cada paso, el sendero serpenteaba a través...   
4  La melodía suave del piano llenaba la sala, cr...   

                                          INCORRECTA  
0  Puesta sol horizonte pintar cielo tonos cálido...  
1  Después una larga jornada trabajo, disfruto re...  
2  Aroma fresco flores primaverales impregnar jar...  
3  Con cada paso, sendero serpenteaba través bosq...  
4  Melodía suave piano llenar sala, creando atmós...   

Index(['CORRECTA', 'INCORRECTA'], dtype='object')


In [57]:
# Función para corregir la gramática de una oración
def corregir_gramatica(oracion):
    tool = language_tool_python.LanguageToolPublicAPI('es')
    matches = tool.check(oracion)

    # Aplicar correcciones sugeridas
    oracion_corregida = tool.correct(oracion)

    return oracion_corregida

# Función para corregir la sintaxis de una oración
def corregir_sintaxis(oracion, modelo_sintaxis, tokenizer, max_length):
    # Preprocesar la oración
    oracion_procesada = tokenizer.texts_to_sequences([oracion])
    input_sequence = pad_sequences(oracion_procesada, padding='post', maxlen=max_length)

    # Obtener predicciones del modelo de corrección sintáctica
    predicciones = modelo_sintaxis.predict(input_sequence)

    # Convertir las predicciones a texto
    predicciones_texto = " ".join([tokenizer.index_word[idx] for idx in np.argmax(predicciones[0], axis=-1)])

    return predicciones_texto

# Función para preprocesar datos
def preprocesar_datos(oraciones_correctas, oraciones_incorrectas):
    oraciones = oraciones_correctas + oraciones_incorrectas
    tokenizer = Tokenizer(oov_token="<OOV>")
    tokenizer.fit_on_texts(oraciones)

    secuencias = tokenizer.texts_to_sequences(oraciones)
    secuencias_padded = pad_sequences(secuencias, padding='post')

    vocab_size = len(tokenizer.word_index) + 1

    # Etiquetas del conjunto de datos (1 para oraciones correctas, 0 para incorrectas)
    etiquetas = np.concatenate([np.ones(len(oraciones_correctas)), np.zeros(len(oraciones_incorrectas))])

    return secuencias_padded, etiquetas, vocab_size, tokenizer

# Función para diseñar el modelo
def diseñar_modelo(vocab_size, max_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
        LSTM(128, return_sequences=True),
        Dense(1, activation='sigmoid')  # Se utiliza 'sigmoid' y 1 nodo para la clasificación binaria
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Función para entrenar el modelo
def entrenar_modelo(model, input_sequences, etiquetas, epochs=10):
    model.fit(input_sequences, etiquetas, epochs=epochs, verbose=2)

# Función para evaluar el modelo
def evaluar_modelo(model, input_sequence, tokenizer):
    predicciones = model.predict(input_sequence)

    # Convertir predicciones a texto
    predicciones_texto = " ".join([tokenizer.index_word[idx] for idx in np.argmax(predicciones[0], axis=-1)])

    return predicciones_texto


In [58]:
# oración a corregir
oracion_original = "Ayer, fui al parque a jugar con mis amigos y me olvidé mis sapatos en casa."


In [59]:

# Oraciones para el entrenamiento
oraciones_correctas = df['CORRECTA'].tolist()
oraciones_incorrectas = df['INCORRECTA'].tolist()

In [60]:

# Preprocesar datos
input_sequences, etiquetas, vocab_size, tokenizer = preprocesar_datos(oraciones_correctas, oraciones_incorrectas)

# Diseñar modelo
max_length = len(input_sequences[0])
modelo_correccion_sintaxis = diseñar_modelo(vocab_size, max_length)

# Entrenar modelo
entrenar_modelo(modelo_correccion_sintaxis, input_sequences, etiquetas, epochs=100)

# Corregir gramática
oracion_corregida_gramatica = corregir_gramatica(oracion_original)

# Corregir sintaxis
oracion_corregida_sintaxis = corregir_sintaxis(oracion_corregida_gramatica, modelo_correccion_sintaxis, tokenizer, max_length)

# Imprimir resultados
print("Oración Original:", oracion_original)
print("Oración Corregida Gramaticalmente:", oracion_corregida_gramatica)
print("Oración Corregida Sintácticamente:", oracion_corregida_sintaxis)

# Calcular accuracy
accuracy = calcular_accuracy(modelo_correccion_sintaxis, input_sequences, etiquetas)
print("Accuracy del modelo:", accuracy)

Epoch 1/100


ValueError: in user code:

    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/sergiogonzalez/anaconda3/lib/python3.11/site-pac